In [1]:
!pip install tensorflow numpy pandas tqdm scikit-learn segmentation-models tensorflow-addons flammkuchen opencv-python-headless imageio

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import tensorflow as tf
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score

from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.data import load_data, MAPE_V1, mape_apV1, mape_ppV1
from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.Callbacks import get_callbacks


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Segmentation Models: using `tf.keras` framework.


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
import cv2
import numpy as np
import imageio as io
from tensorflow.keras.models import load_model
import flammkuchen as fl
from google.colab.patches import cv2_imshow
import flammkuchen as fl
import tensorflow_addons as tfa


# Función para dibujar en el contorno de la mascara en la imagen original
def draw_contours_on_image(mask, img_orig, contour_color=(255, 255, 255)):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img_with_contours = img_orig.copy()
    cv2.drawContours(img_with_contours, contours, -1, contour_color, 2)
    return img_with_contours

# Función para dibujar los puntos y la línea en la imagen
def draw_points_and_line(img, point1, point2, color=(0, 255, 0), thickness=2):
    cv2.circle(img, point1, radius=5, color=color, thickness=-1)
    cv2.circle(img, point2, radius=5, color=color, thickness=-1)
    cv2.line(img, point1, point2, color=color, thickness=thickness)
    return img

# Función para dibujar los contornos de la segmentación en la imagen
def draw_contours_on_image(mask, img, contour_color=(255, 0, 0)):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, contour_color, 2)
    return img

# Función que integra todo el flujo de trabajo
def process_and_recreate_video(video_path, model_path, output_video_path):
    model = load_model(model_path, compile=False,
                      custom_objects={'InstanceNormalization': tfa.layers.InstanceNormalization})

    ims = io.mimread(video_path, memtest=False)

    # Preparar el objeto VideoWriter
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (ims[0].shape[1], ims[0].shape[0]))

    # Procesar cada frame del video
    for frame in ims:
        img_orig = frame.astype(np.float32)
        img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img_orig, (256, 512))

        # Normalize and preprocess image
        normalizedImg = np.zeros(img.shape)
        img = cv2.normalize(img, normalizedImg, -1, 1, cv2.NORM_MINMAX)
        img = img[None, ..., None]
        ap_pred, seg_pred = model.predict(img)

        # Extraer los puntos de máxima activación en cada mapa de calor
        point1_y, point1_x = np.unravel_index(np.argmax(ap_pred[0, :, :, 0]), (512, 256))
        point2_y, point2_x = np.unravel_index(np.argmax(ap_pred[0, :, :, 1]), (512, 256))

        # Escalar las coordenadas al tamaño original de la imagen
        frame_height, frame_width = frame.shape[:2]
        point1 = (int(frame_width * (point1_x / 256)), int(frame_height * (point1_y / 512)))
        point2 = (int(frame_width * (point2_x / 256)), int(frame_height * (point2_y / 512)))

        # Convertir la predicción de segmentación a una máscara binaria y dibujar los contornos y los puntos
        seg_mask = (seg_pred[0, :, :, 0] > 0.5).astype(np.uint8)
        img_with_contours = draw_contours_on_image(seg_mask * 255, frame, contour_color=(255, 0, 0))
        img_with_points_and_line = draw_points_and_line(img_with_contours, point1, point2)
        video_writer.write(img_with_points_and_line)

    # Liberar recursos
    video_writer.release()
    cap.release()



In [6]:
model_path = 'drive/MyDrive/Glottis/GlottisNetV1/Model/steps/epoch020.h5'  # Ajusta según tu path
video_path = 'drive/MyDrive/Glottis/videos_BAGLS/40.mp4'  # Ajusta según tu path
mask_save_path = 'drive/MyDrive/Glottis/GlottisNetV1/masks/40.mask'  # Ajusta según tu path
output_video_path = '/content/drive/MyDrive/Glottis/videos_segmentados/40_v1.mp4'

process_and_recreate_video(video_path, model_path, output_video_path)

1/1 [==============================] - 1s 866ms/step
